In [55]:
import requests
import pandas as pd

In [56]:
def consultar(pais: str, indicador:str, pagina:int = 1, iyear:int = 1990, fyear:int = 2022):
    '''Función que contacta a la API y devuelve la respuesta
    que brinda la misma en formato json'''

    # Página de la api y path al recurso solicitado
    api_url = 'http://api.worldbank.org/v2/es'
    path = f'/country/{pais}/indicator/{indicador}'
    url = api_url + path

    # Creamos el diccionario con los parametros 
    # para el método get
    args= {
        "date":str(iyear)+":"+str(fyear),
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
   
    return requests.get(url,params=args).json()

In [57]:
def consultar_por_pais():
    '''Función que retorna un código de país,
    para, en caso que se quiera o se solicite,
    consultar en tiempo real la API'''
    paises=pd.read_parquet('../datasets/paises.parquet')

    seleccion = input("Ingrese un nombre de país: ")
    try:
        pais = paises.loc[paises.name.str.contains(seleccion, case=False),'id'].values[0]
    except:
        print("No se encontró el país, intente de nuevo")

In [58]:
# Estos indicadores fueron seleccionados 
# de forma manual al final del documento
indicadores = pd.read_csv('../datasets/indicadores_seleccionados.csv')
indicadores.head()

,Unnamed: 0,id,name,sourceNote
0,0,SP.DYN.LE00.IN,"Esperanza de vida al nacer, total (años)",La esperanza de vida al nacer indica la cantid...
1,1,SP.DYN.LE00.FE.IN,"Esperanza de vida al nacer, mujeres (años)",La esperanza de vida al nacer indica la cantid...
2,1,SP.DYN.LE00.MA.IN,"Esperanza de vida al nacer, varones (años)",La esperanza de vida al nacer indica la cantid...
3,3,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...
4,4,SE.XPD.TOTL.GD.ZS,"Gasto público en educación, total (% del PIB)",El gasto público en educación como porcentaje ...


In [59]:
tuplas = [(a,b) for a,b in zip(indicadores.id.to_list(), 
          indicadores.name.to_list())]

In [60]:
def carga_incremental(pais, indicador):
    '''Función que a partir de un país y un indicador 
    llama a consultar y establece qué tipo de contenido tiene
    según eso devuelve o no un dataframe con todos los datos'''


    consulta = consultar(pais, indicador)
    try:
        # La primera parte de la respuesta nos indica en 
        # cuantas páginas de encuentra la información
        paginas = consulta[0]["pages"]

        # La segunda parte nos retorna una lista de 
        # diccionarios con la información que queríamos
        datos=consulta[1]

    except:
        print('No hay datos para:', indicador, pais)
        pass
    else:
        if paginas >= 1:
            # Agregamos los valores de las otras páginas a
            # nuestra lista de diccionarios
            for pagina in range(2,paginas+1):
                datos.extend(consultar(pais, indicador, pagina)[1])

            # Creo el DataFrame con todos los datos
            data = pd.json_normalize(datos)
            
            # Me quedo con información relevante unicamente
            #data = data[['countryiso3code','country.value','date','value']]
            #data.rename(columns={'value':nombre},inplace=True)
        
        return data


In [62]:
for indicador, _ in tuplas:
    print(indicador)

    datos = carga_incremental(pais='all', indicador=indicador)

    # Guardo el dataframe resultante
    datos.to_parquet(f'../datasets/df_TWB_{indicador}.parquet')


SP.DYN.LE00.IN
SP.DYN.LE00.FE.IN
SP.DYN.LE00.MA.IN
3.0.Gini
SE.XPD.TOTL.GD.ZS
SE.COM.DURS
6.0.GDP_current
NY.GDP.PCAP.CD
NY.GDP.MKTP.PP.CD
IQ.SCI.OVRL
SP.POP.TOTL.FE.ZS
SP.POP.TOTL.MA.ZS


Función para agregar todos los indicadores a un dataframe y proceder al análisis de características

In [ ]:
def merge_de_características():
    '''Función para unir las diferentes características
    de diferentes dataframes en uno solo'''
    indicador, nombre = tuplas.pop(0)
    datos = carga_incremental(pais, indicador, nombre)

    for indicador, nombre in tuplas:
        for pais in paises:
            print(indicador)
            try:
                a_sumar = carga_incremental(indicador, nombre)
                a_sumar = a_sumar[['date', nombre]]
            except TypeError:
                pass
            else:
                datos = datos.merge(a_sumar, how='left', on='date')

# ETL

## Búsqueda de indicadores

Procedo a armar la lista de indicadores que estarán bajo estudio

In [54]:
trabajo = pd.DataFrame()

In [47]:
indicators = pd.read_csv('../datasets/indicadores.csv')

# Se completó los valores faltantes con 'na'
# para poder buscar sobre el dataframe
indicators.fillna('na',inplace=True)

Buscamos de forma manual el in

In [13]:
interes = input("Indicador de interés: ")
indicators.loc[indicators.name.str.contains(interes,case=False),
     ['id','name','sourceNote']]#.to_csv('temp.csv')

,id,name,sourceNote
179,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...
180,3.0.Gini_nozero,Coeficiente de Gini (Ingreso diferente de cero),El coeficiente de Gini es la medida de desigua...
192,3.1.Gini,"Gini, Rural",El coeficiente de Gini es la medida de desigua...
205,3.2.Gini,"Gini, Urbano",El coeficiente de Gini es la medida de desigua...
17088,SI.POV.GINI,Índice de Gini,El índice de Gini mide hasta qué punto la dist...


In [14]:
interes = input("Indicador de interés: ")
pick = indicators.loc[indicators.id == interes,
     ['id','name','sourceNote']]
pick

,id,name,sourceNote
179,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...


In [16]:
trabajo = pd.concat([trabajo, pick], ignore_index=True)

In [17]:
trabajo#.to_csv('../datasets/indicadores_seleccionados.csv')

,id,name,sourceNote
0,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...
